In [1]:
import pandas as pd
import numpy as np
import re

In [4]:
path = "../../dataVault/waste_edge_booking_data/27.1.2021_2.2.2021.csv"
# 27.1.2021_2.2.2021
df = pd.read_csv(path, dtype={"Schd Time Start" : str, "PO" : str})

In [5]:
df['Route number'] = df['Route number'].astype('str')
df[['Route number', 'weekday']] = df['Route number'].str.split('-',1,expand=True)

In [27]:
def rev_type_hardcode(rev_type: str):
        switcher = {
            'total': 'total',
            'general_waste': ['HOOK1', 'BR1', 'BR2', 'BR3', 'FL2', 'FLG', 'RL1', 'RL2', 'RL4', 'RL7', 'RL9', 'RLD', 'RLE', 'RLH', 'RLI', 'RLJ', 'RLK', 'SWG', 'AUSSKIP'],
            'cardboard': ['GRIMA', 'APR', 'FLP', 'HYG', 'RED', 'RL5', 'RL6', 'RL8', 'RLP', 'RLR', 'SWP'],
            'comingled': ['CBK', 'RLC', 'RLG', 'DOY'],
            'subContractor': ['SUB', 'JJT', 'ALLMED', 'BIN', 'CKG', 'CLN', 'GRACE', 'JJR', 'OWE', 'REM', 'REP', 'REQ', 'RRNW', 'RRR', 'SHR', 'SPD', 'SUE', 'URM', 'VEO', 'VEOACT', 'VTG'],
            'uos': ['NEPCB', 'UOSCB', 'UOSCO', 'UOSGW', 'CMDCB', 'CMDGW', 'CUMCB', 'CUMGW', 'NEPGW']
        }
        rev_routes = switcher.get(rev_type, "invalid entry")
        return rev_routes


In [41]:
df['Date'] = pd.to_datetime(df['Date'],format='%d/%m/%y')
df['Date'] = pd.DatetimeIndex(df['Date'])
df.set_index(df['Date'], inplace=True)
df['Date'].dtype

subContractor = rev_type_hardcode('subContractor')
uos = rev_type_hardcode('uos')

df_weekly_groups = df.resample("7D")
date_key = list(df_weekly_groups.groups.keys())

current_week_df = df_weekly_groups.get_group(date_key[0])
uos_rows = current_week_df['Route number'].isin(uos)

current_week_df[uos_rows]


result = (current_week_df
     .pipe(lambda data : data.groupby('Route number').Price.sum())
     .pipe(lambda data : data.filter(uos))
#      .pipe(lambda data : data.sum())
)

result 



Route number
NEPCB        1.14
UOSCB     1332.60
UOSCO      771.49
UOSGW    19367.79
CMDCB       30.87
CMDGW     1152.41
CUMCB        6.86
CUMGW      817.10
NEPGW       73.61
Name: Price, dtype: float64

In [ ]:
# df_weekly_groups.get_group(df_weekly_groups_keys[0])


# df_weekly_groups_keys = list(df_weekly_groups_keys)
# df_first_week = df.resample("7D").get_group(df_weekly_groups_keys[0])

# df_first_week

# (df_first_week
#      .groupby('Route number')
#      .pipe(lambda x : x.Price.sum())
#      .pipe(lambda x : x.filter(items=uos))
#      .pipe(lambda x : x.sum())
# )



# 

# df.groupby('group')
#    .pipe(f)
#    .pipe(g, arg1=a)
#    .pipe(h, arg2=b, arg3=c))  


# After date time index it must sort by sort_index
# sort_index

# .reset_index opposite to set_index 


# df.sort_values(by=['Date'], inplace=True, ascending=False)
# df.resample("7D")
# route_num_groups = df.groupby('Route number').Price.sum()
# uos = rev_type_hardcode('uos')
# sub = rev_type_hardcode('subContractor')
# route_num_groups.filter(items=uos).getgroup

# Start time
# origin{‘epoch’, ‘start’, ‘start_day’}, Timestamp or str, default ‘start_day’
# The timestamp on which to adjust the grouping. The timezone of origin must match the timezone of the index. If a timestamp is not used, these values are also supported:

# ‘epoch’: origin is 1970-01-01

# ‘start’: origin is the first value of the timeseries

# ‘start_day’: origin is the first day at midnight of the timeseries

# New in version 1.1.0.